In [1]:
from pathlib import Path
import pandas as pd

In [2]:
# Базовые пути
BASE = Path("..").resolve()
RAW_BASE = BASE / "data" / "raw" / "fbref"

SEASONS = ["epl_2019-2020","epl_2020-2021","epl_2021-2022","epl_2022-2023","epl_2023-2024","epl_2024-2025"]
FILES = ["schedule_results.csv", "standings.csv", "team_standard_stats.csv"]

In [3]:
# Быстрая проверка наличия файлов по каждому сезону
summary = []
for s in SEASONS:
    d = RAW_BASE / s
    row = {"season": s}
    for f in FILES:
        p = d / f
        row[f] = "OK" if p.exists() else "MISSING"
        if p.exists():
            try:
                df = pd.read_csv(p, nrows=2)
                row[f + "_cols"] = df.shape[1]
            except Exception as e:
                row[f] = f"READ_ERR: {e}"
    summary.append(row)

pd.DataFrame(summary)

,season,schedule_results.csv,schedule_results.csv_cols,standings.csv,standings.csv_cols,team_standard_stats.csv,team_standard_stats.csv_cols
0,epl_2019-2020,OK,14,OK,19,OK,32
1,epl_2020-2021,OK,14,OK,19,OK,32
2,epl_2021-2022,OK,14,OK,19,OK,32
3,epl_2022-2023,OK,14,OK,19,OK,32
4,epl_2023-2024,OK,14,OK,19,OK,32
5,epl_2024-2025,OK,14,OK,19,OK,32


In [4]:
all_standings = []
all_teamstats = []

for s in SEASONS:
    d = RAW_BASE / s
    try:
        st = pd.read_csv(d / "standings.csv")
        st["season"] = s
        all_standings.append(st)
    except Exception as e:
        print(f"[ERR] standings {s}: {e}")

    try:
        ts = pd.read_csv(d / "team_standard_stats.csv")
        ts["season"] = s
        all_teamstats.append(ts)
    except Exception as e:
        print(f"[ERR] team stats {s}: {e}")

standings_all = pd.concat(all_standings, ignore_index=True)
teamstats_all = pd.concat(all_teamstats, ignore_index=True)

print("✅ Объединение завершено!")
print("Standings shape:", standings_all.shape)
print("Team stats shape:", teamstats_all.shape)

✅ Объединение завершено!
Standings shape: (120, 20)
Team stats shape: (120, 33)


In [5]:
import pandas as pd

# Проверим первые строки и столбцы для одного сезона (например, 2022-2023)
season_check = "epl_2022-2023"
check_path = RAW_BASE / season_check

# Загружаем фикстуры
fx = pd.read_csv(check_path / "schedule_results.csv")

print(f"📅 {season_check} — schedule_results.csv")
print(f"Размер: {fx.shape}")
print("Столбцы:", list(fx.columns))
display(fx.head(3))

📅 epl_2022-2023 — schedule_results.csv
Размер: (392, 14)
Столбцы: ['wk', 'day', 'date', 'time', 'home', 'xg', 'score', 'xg_1', 'away', 'attendance', 'venue', 'referee', 'match_report', 'notes']


,wk,day,date,time,home,xg,score,xg_1,away,attendance,venue,referee,match_report,notes
0,1,Fri,2022-08-05,20:00 (21:00),Crystal Palace,1.2,0–2,1.0,Arsenal,25286,Selhurst Park,Anthony Taylor,Match Report,NaN
1,1,Sat,2022-08-06,12:30 (13:30),Fulham,1.2,2–2,1.2,Liverpool,22207,Craven Cottage,Andy Madley,Match Report,NaN
2,1,Sat,2022-08-06,15:00 (16:00),Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245,St James' Park,Simon Hooper,Match Report,NaN


In [6]:
from collections import Counter

cols_per_season = {}
for s in SEASONS:
    path = RAW_BASE / s / "schedule_results.csv"
    df = pd.read_csv(path)
    cols_per_season[s] = list(df.columns)

# Проверим уникальные наборы колонок
unique_structures = Counter(tuple(cols) for cols in cols_per_season.values())
print(f"Уникальных структур: {len(unique_structures)}")

# Если всё ок, выведет 1
if len(unique_structures) == 1:
    print("✅ Все сезоны имеют одинаковую структуру fixtures.csv")
else:
    print("⚠️ Различия найдены — нужно проверить вручную")
    for k, v in cols_per_season.items():
        print(f"{k}: {v}")

Уникальных структур: 1
✅ Все сезоны имеют одинаковую структуру fixtures.csv


In [7]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

# ожидается, что SEASONS и RAW_BASE уже определены в ноутбуке
PROCESSED_DIR = (RAW_BASE.parent.parent / "processed" / "fbref")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "epl_fixtures_all.csv"

def parse_score_to_cols(series):
    """Из '2–1' / '2-1' -> два столбца (home_goals, away_goals)."""
    def f(s):
        if pd.isna(s):
            return np.nan, np.nan
        m = re.search(r"(\d+)\s*[–-]\s*(\d+)", str(s))
        return (int(m.group(1)), int(m.group(2))) if m else (np.nan, np.nan)
    return series.apply(lambda s: pd.Series(f(s), index=["home_goals","away_goals"]))

frames = []
for s in SEASONS:
    df = pd.read_csv(RAW_BASE / s / "schedule_results.csv")
    df["season"] = s
    # унифицируем имена на всякий случай
    df.columns = (
        df.columns.str.strip().str.lower()
          .str.replace(" ", "_").str.replace("%","pct")
          .str.replace("/", "_").str.replace("-", "_", regex=False)
    )
    # распарсим счёт
    if "score" in df.columns:
        parsed = parse_score_to_cols(df["score"])
        df = pd.concat([df, parsed], axis=1)
    frames.append(df)

fixtures_all = pd.concat(frames, ignore_index=True)

# оставим только сыгранные матчи (есть голы у обеих сторон)
fixtures_all = fixtures_all.dropna(subset=["home_goals","away_goals"]).copy()

# аккуратно приведём типы
for col in ["home_goals","away_goals"]:
    fixtures_all[col] = fixtures_all[col].astype(int)

# порядок столбцов (если есть)
cols_order = [c for c in ["season","wk","date","time","home","away","score","home_goals","away_goals","xg","xg_1","venue","referee","attendance"] if c in fixtures_all.columns]
fixtures_all = fixtures_all[[*cols_order, *[c for c in fixtures_all.columns if c not in cols_order]]]

# сохранение
fixtures_all.to_csv(out_path, index=False, encoding="utf-8")

print(f"✅ Объединено фикстур: {fixtures_all.shape[0]} строк, {fixtures_all.shape[1]} столбцов")
print(f"📁 Сохранено в: {out_path}")
fixtures_all.head(3)

✅ Объединено фикстур: 2280 строк, 17 столбцов
📁 Сохранено в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/fbref/epl_fixtures_all.csv


,season,wk,date,time,home,away,score,home_goals,away_goals,xg,xg_1,venue,referee,attendance,day,match_report,notes
0,epl_2019-2020,1,2019-08-09,20:00 (21:00),Liverpool,Norwich City,4–1,4,1,1.8,0.9,Anfield,Michael Oliver,53333,Fri,Match Report,NaN
1,epl_2019-2020,1,2019-08-10,12:30 (13:30),West Ham,Manchester City,0–5,0,5,1.1,3.2,London Stadium,Mike Dean,59870,Sat,Match Report,NaN
2,epl_2019-2020,1,2019-08-10,15:00 (16:00),Bournemouth,Sheffield Utd,1–1,1,1,1.3,1.3,Vitality Stadium,Kevin Friend,10714,Sat,Match Report,NaN


In [8]:
# === Объединяем standings по всем сезонам ===
frames = []
for s in SEASONS:
    path = RAW_BASE / s / "standings.csv"
    df = pd.read_csv(path)
    df["season"] = s
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    frames.append(df)

standings_all = pd.concat(frames, ignore_index=True)

# аккуратно приведём типы
for c in ["w", "d", "l", "gf", "ga", "gd", "pts"]:
    if c in standings_all.columns:
        standings_all[c] = pd.to_numeric(standings_all[c], errors="coerce")

# сохраним
standings_out = PROCESSED_DIR / "epl_standings_all.csv"
standings_all.to_csv(standings_out, index=False, encoding="utf-8")

print(f"✅ Объединено standings: {standings_all.shape[0]} строк, {standings_all.shape[1]} столбцов")
print(f"📁 Сохранено в: {standings_out}")
standings_all.head(3)

✅ Объединено standings: 120 строк, 20 столбцов
📁 Сохранено в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/fbref/epl_standings_all.csv


,rk,squad,mp,w,d,l,gf,ga,gd,pts,pts_mp,xg,xga,xgd,xgd_90,attendance,top_team_scorer,goalkeeper,notes,season
0,1,Liverpool,38,32,3,3,85,33,52,99,2.61,68.9,37.7,31.2,0.82,41955,Mohamed Salah - 19,Alisson,→ Champions League via league finish,epl_2019-2020
1,2,Manchester City,38,26,3,9,102,35,67,81,2.13,92.0,36.2,55.7,1.47,37097,Raheem Sterling - 20,Ederson,→ Champions League via league finish,epl_2019-2020
2,3,Manchester Utd,38,18,12,8,66,36,30,66,1.74,62.2,37.7,24.5,0.64,57415,"Anthony Martial, Marcus Rashford - 17",David de Gea,→ Champions League via league finish,epl_2019-2020


In [9]:
# === Объединяем team_standard_stats по всем сезонам ===
from pathlib import Path
import pandas as pd

PROCESSED_DIR = (Path("..").resolve() / "data" / "processed" / "fbref")
RAW_BASE = (Path("..").resolve() / "data" / "raw" / "fbref")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

SEASONS = ["epl_2019-2020","epl_2020-2021","epl_2021-2022","epl_2022-2023","epl_2023-2024","epl_2024-2025"]

frames = []
for s in SEASONS:
    p = RAW_BASE / s / "team_standard_stats.csv"
    df = pd.read_csv(p)
    df["season"] = s
    # унифицируем имена
    df.columns = (
        df.columns.str.strip().str.lower()
          .str.replace(" ", "_").str.replace("%","pct")
          .str.replace("/", "_").str.replace("-", "_", regex=False)
    )
    # приводим числовые
    num_like = df.columns.difference(["squad","season"])
    df[num_like] = df[num_like].apply(pd.to_numeric, errors="coerce")
    frames.append(df)

team_stats_all = pd.concat(frames, ignore_index=True)

# быстрая проверка целостности
expected_rows = 20 * len(SEASONS)
print(f"✅ team_stats: {team_stats_all.shape} (ожидали ~{expected_rows} строк)")
dups = team_stats_all.duplicated(subset=["season","squad"]).sum()
print(f"🔎 Дубликаты по (season,squad): {dups}")

out_path = PROCESSED_DIR / "epl_team_stats_all.csv"
team_stats_all.to_csv(out_path, index=False, encoding="utf-8")
print(f"📁 Сохранено в: {out_path}")

team_stats_all.head(3)

✅ team_stats: (120, 33) (ожидали ~120 строк)
🔎 Дубликаты по (season,squad): 0
📁 Сохранено в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/fbref/epl_team_stats_all.csv


,squad,pl,age,poss,mp,starts,min,90s,gls,ast,...,ast_per90,g_plus_a_per90,g_pk_per90,g_plus_a_pk_per90,xg_per90,xag_per90,xg_xag_per90,npxg_per90,npxg_xag_per90,season
0,Arsenal,29,25.8,53.8,38,418,3420,38.0,56,35,...,0.92,2.39,1.39,2.32,1.24,0.83,2.06,1.17,2.0,epl_2019-2020
1,Aston Villa,28,25.7,44.1,38,418,3420,38.0,40,32,...,0.84,1.89,1.03,1.87,1.16,0.90,2.06,1.10,2.0,epl_2019-2020
2,Bournemouth,27,25.2,44.1,38,418,3420,38.0,38,24,...,0.63,1.63,0.89,1.53,1.18,0.80,1.98,1.10,1.9,epl_2019-2020


In [10]:
# === Объединяем standings и team stats ===

import pandas as pd
from pathlib import Path

PROCESSED_DIR = Path("..").resolve() / "data" / "processed" / "fbref"
standings_all = pd.read_csv(PROCESSED_DIR / "epl_standings_all.csv")
team_stats_all = pd.read_csv(PROCESSED_DIR / "epl_team_stats_all.csv")

# Убираем возможные пробелы и приводим ключи к единому виду
standings_all["squad"] = standings_all["squad"].str.strip()
team_stats_all["squad"] = team_stats_all["squad"].str.strip()

merged = standings_all.merge(team_stats_all, on=["season", "squad"], how="left", suffixes=("_stand", "_team"))
print(f"✅ Объединено: {merged.shape[0]} строк, {merged.shape[1]} столбцов")

merged_path = PROCESSED_DIR / "epl_full_dataset.csv"
merged.to_csv(merged_path, index=False, encoding="utf-8")
print(f"📁 Сохранено в: {merged_path}")

merged.head(3)

✅ Объединено: 120 строк, 51 столбцов
📁 Сохранено в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/fbref/epl_full_dataset.csv


,rk,squad,mp_stand,w,d,l,gf,ga,gd,pts,...,gls_per90,ast_per90,g_plus_a_per90,g_pk_per90,g_plus_a_pk_per90,xg_per90,xag_per90,xg_xag_per90,npxg_per90,npxg_xag_per90
0,1,Liverpool,38,32,3,3,85,33,52,99,...,2.18,1.82,4.00,2.05,3.87,1.81,1.39,3.21,1.71,3.10
1,2,Manchester City,38,26,3,9,102,35,67,81,...,2.63,1.82,4.45,2.47,4.29,2.42,1.80,4.22,2.20,4.00
2,3,Manchester Utd,38,18,12,8,66,36,30,66,...,1.71,1.13,2.84,1.45,2.58,1.64,1.01,2.65,1.35,2.36


In [11]:
# === Проверка качества объединённого датасета ===

import pandas as pd
from pathlib import Path

DATASET_PATH = Path("..") / "data" / "processed" / "fbref" / "epl_full_dataset.csv"
df = pd.read_csv(DATASET_PATH)

print(f"✅ Загружено: {df.shape[0]} строк, {df.shape[1]} столбцов\n")

# 1️⃣ Проверка уникальности ключей
dupes = df.duplicated(subset=["season", "squad"]).sum()
print(f"🔎 Дубликаты по (season, squad): {dupes}")

# 2️⃣ Пропуски
na_counts = df.isna().sum()
na_cols = na_counts[na_counts > 0]
print(f"\n⚠️ Пропуски найдены в {len(na_cols)} колонках:")
display(na_cols.sort_values(ascending=False))

# 3️⃣ Типы данных
print("\n📊 Типы данных:")
display(df.dtypes.head(10))

# 4️⃣ Быстрая описательная статистика
display(df.describe().T.head(10))

✅ Загружено: 120 строк, 51 столбцов

🔎 Дубликаты по (season, squad): 0

⚠️ Пропуски найдены в 1 колонках:


notes    56
dtype: int64


📊 Типы данных:


rk           int64
squad       object
mp_stand     int64
w            int64
d            int64
l            int64
gf           int64
ga           int64
gd           int64
pts          int64
dtype: object

,count,mean,std,min,25%,50%,75%,max
rk,120.0,10.500000,5.790459,1.00,5.75,10.50,15.2500,20.00
mp_stand,120.0,38.000000,0.000000,38.00,38.00,38.00,38.0000,38.00
w,120.0,14.625000,6.249790,2.00,11.00,14.00,19.0000,32.00
d,120.0,8.750000,2.855777,2.00,7.00,8.50,10.0000,15.00
l,120.0,14.625000,6.082158,2.00,11.00,14.00,18.2500,30.00
gf,120.0,54.783333,17.730562,20.00,41.00,54.00,66.2500,102.00
ga,120.0,54.783333,14.201541,26.00,45.75,54.00,63.2500,104.00
gd,120.0,0.000000,28.599524,-69.00,-18.25,-1.00,15.0000,73.00
pts,120.0,52.525000,18.205162,12.00,40.00,52.00,65.2500,99.00
pts_mp,120.0,1.382333,0.479157,0.32,1.05,1.37,1.7175,2.61
